## Feature Engineering

In [1]:
import numpy as np
import pandas as pd

order_new=pd.read_csv('/Users/jinglinhui/Desktop/Data_analytics_project/rawdata/orders_new.csv')
order_product_new=pd.read_csv('/Users/jinglinhui/Desktop/Data_analytics_project/rawdata/order_products_new.csv')

In [2]:
order_new = order_new.fillna(0)

In [3]:
# join orders_new and order_product_new together
join = pd.merge(order_product_new, order_new, on = 'order_id', how = 'left' )
join.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,eval_set
0,2,33120,1,1,202279,3,5,9,8.0,prior
1,2,28985,2,1,202279,3,5,9,8.0,prior
2,2,9327,3,0,202279,3,5,9,8.0,prior
3,2,45918,4,1,202279,3,5,9,8.0,prior
4,2,30035,5,0,202279,3,5,9,8.0,prior


#### 1. order_product_prior

In [4]:
order_product_prior = join[join['eval_set'] == 'prior']
order_product_prior.shape

(32434489, 10)

In [5]:
order_product_prior.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,eval_set
0,2,33120,1,1,202279,3,5,9,8.0,prior
1,2,28985,2,1,202279,3,5,9,8.0,prior
2,2,9327,3,0,202279,3,5,9,8.0,prior
3,2,45918,4,1,202279,3,5,9,8.0,prior
4,2,30035,5,0,202279,3,5,9,8.0,prior


#### 2. user_feature_1

In [6]:
df1 = order_new[order_new['eval_set'] == 'prior']

user_id = df1['user_id'].unique()
user_orders = df1.groupby(['user_id'])['order_number'].max()
sum_days_since_prior_order = df1.groupby(['user_id'])['days_since_prior_order'].sum()
avg_days_since_prior_order = df1.groupby(['user_id'])['days_since_prior_order'].mean()

In [7]:
user_feature_1 = pd.DataFrame({
    'user_orders': user_orders,
    'sum_days_since_prior_order': sum_days_since_prior_order,
    'avg_days_since_prior_order': avg_days_since_prior_order
})
user_feature_1 = user_feature_1.reset_index()
user_feature_1.head(10)

,user_id,user_orders,sum_days_since_prior_order,avg_days_since_prior_order
0,1,10,176.0,17.600000
1,2,14,198.0,14.142857
2,3,12,133.0,11.083333
3,4,5,55.0,11.000000
4,5,4,40.0,10.000000
5,6,3,18.0,6.000000
6,7,20,203.0,10.150000
7,8,3,60.0,20.000000
8,9,3,36.0,12.000000
9,10,5,79.0,15.800000


#### 03. user_feature_2

In [8]:
df = order_product_prior[order_product_prior['order_number'] > 1]

total_products = order_product_prior.groupby('user_id')['product_id'].count()
distinct_products = order_product_prior.groupby('user_id')['product_id'].nunique()
reordered_sum = order_product_prior.groupby('user_id')['reordered'].sum()
reordered_count = df.groupby('user_id')['reordered'].count()

In [9]:
user_feature_2 = pd.DataFrame({
    'total_products': total_products,
    'distinct_products': distinct_products,
    'reordered_ratio': reordered_sum/reordered_count
})
user_feature_2.head(10)

,total_products,distinct_products,reordered_ratio
user_id,,,
1,59,18,0.759259
2,195,102,0.510989
3,88,33,0.705128
4,18,17,0.071429
5,37,23,0.538462
6,14,12,0.200000
7,206,68,0.711340
8,49,36,0.464286
9,76,58,0.391304


In [10]:
user_feature_2 = user_feature_2.reset_index()
user_feature_2.head(10)

,user_id,total_products,distinct_products,reordered_ratio
0,1,59,18,0.759259
1,2,195,102,0.510989
2,3,88,33,0.705128
3,4,18,17,0.071429
4,5,37,23,0.538462
5,6,14,12,0.200000
6,7,206,68,0.711340
7,8,49,36,0.464286
8,9,76,58,0.391304
9,10,143,94,0.355072


#### 04. up_feature

In [11]:
m = order_product_prior.groupby(['user_id','product_id'])

In [12]:
up_feature = pd.DataFrame({
    'up_orders': m['order_number'].count(),
    'up_first_order': m['order_number'].min(),
    'up_last_order': m['order_number'].max(),
    'up_average_cart_position': m['add_to_cart_order'].mean()
})

In [13]:
up_feature = up_feature.reset_index()
up_feature.head(10)

,user_id,product_id,up_orders,up_first_order,up_last_order,up_average_cart_position
0,1,196,10,1,10,1.400000
1,1,10258,9,2,10,3.333333
2,1,10326,1,5,5,5.000000
3,1,12427,10,1,10,3.300000
4,1,13032,3,2,10,6.333333
5,1,13176,2,2,5,6.000000
6,1,14084,1,1,1,2.000000
7,1,17122,1,5,5,6.000000
8,1,25133,8,3,10,4.000000
9,1,26088,2,1,2,4.500000


#### 5. prd_feature

In [14]:
order_product_prior['product_seq_time'] = order_product_prior.groupby(['user_id','product_id'])['order_number'].rank()

<ipython-input-14-a4f470e58f8b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  order_product_prior['product_seq_time'] = order_product_prior.groupby(['user_id','product_id'])['order_number'].rank()


In [15]:
m = order_product_prior.groupby('product_id')

In [16]:
prd_feature = pd.DataFrame({
    'prod_orders': m['order_number'].count(),
    'prod_reorders': m['reordered'].sum(),
    'prod_first_orders': order_product_prior[order_product_prior['product_seq_time'] == 1].groupby('product_id')['product_seq_time'].sum(),
    'prod_second_orders': order_product_prior[order_product_prior['product_seq_time'] == 2].groupby('product_id')['product_seq_time'].count()
})

In [17]:
prd_feature = prd_feature.reset_index()
prd_feature.head(10)

,product_id,prod_orders,prod_reorders,prod_first_orders,prod_second_orders
0,1,1852,1136,716.0,276.0
1,2,90,12,78.0,8.0
2,3,277,203,74.0,36.0
3,4,329,147,182.0,64.0
4,5,15,9,6.0,4.0
5,6,8,3,5.0,2.0
6,7,30,12,18.0,6.0
7,8,165,83,82.0,30.0
8,9,156,82,74.0,31.0
9,10,2572,1304,1268.0,399.0


In [18]:
data = pd.merge(user_feature_1, user_feature_2, on = 'user_id', how = 'left')
data = pd.merge(data, up_feature, on = 'user_id', how = 'left')
data = pd.merge(data, prd_feature, on = 'product_id', how = 'left')

In [19]:
data.head(10)

,user_id,user_orders,sum_days_since_prior_order,avg_days_since_prior_order,total_products,distinct_products,reordered_ratio,product_id,up_orders,up_first_order,up_last_order,up_average_cart_position,prod_orders,prod_reorders,prod_first_orders,prod_second_orders
0,1,10,176.0,17.6,59,18,0.759259,196,10,1,10,1.400000,35791,27791,8000.0,4660.0
1,1,10,176.0,17.6,59,18,0.759259,10258,9,2,10,3.333333,1946,1389,557.0,308.0
2,1,10,176.0,17.6,59,18,0.759259,10326,1,5,5,5.000000,5526,3603,1923.0,1003.0
3,1,10,176.0,17.6,59,18,0.759259,12427,10,1,10,3.300000,6476,4797,1679.0,889.0
4,1,10,176.0,17.6,59,18,0.759259,13032,3,2,10,6.333333,3751,2465,1286.0,617.0
5,1,10,176.0,17.6,59,18,0.759259,13176,2,2,5,6.000000,379450,315913,63537.0,45231.0
6,1,10,176.0,17.6,59,18,0.759259,14084,1,1,1,2.000000,15935,12923,3012.0,1895.0
7,1,10,176.0,17.6,59,18,0.759259,17122,1,5,5,6.000000,13880,9377,4503.0,2375.0
8,1,10,176.0,17.6,59,18,0.759259,25133,8,3,10,4.000000,6196,4586,1610.0,935.0
9,1,10,176.0,17.6,59,18,0.759259,26088,2,1,2,4.500000,2523,1360,1163.0,471.0


In [20]:
data['prod_reorder_probability'] = data.prod_second_orders / data.prod_first_orders
#how likely the customer is to purchase the product again in the next order
data['prod_reorder_times'] = 1 + data.prod_reorders / data.prod_first_orders
#number of the times that the products gets reordered again after the first time
data['prod_reorder_ratio'] = data.prod_reorders / data.prod_orders
#possibility that the products will get ordered again by the customer
data.drop(['prod_reorders', 'prod_first_orders', 'prod_second_orders'], axis=1, inplace=True)
data['user_average_basket'] = data.total_products / data.user_orders
#avergae number products in one order
data['up_order_rate'] = data.up_orders / data.user_orders
#how often the customer purchases the products (per order)
data['up_orders_since_last_order'] = data.user_orders - data.up_last_order
#number of orders since last order (per product)
data['up_order_rate_since_first_order'] = data.up_orders / (data.user_orders - data.up_first_order + 1)
#rate of ordering of the product by the customer

In [21]:
data.head(10)

,user_id,user_orders,sum_days_since_prior_order,avg_days_since_prior_order,total_products,distinct_products,reordered_ratio,product_id,up_orders,up_first_order,up_last_order,up_average_cart_position,prod_orders,prod_reorder_probability,prod_reorder_times,prod_reorder_ratio,user_average_basket,up_order_rate,up_orders_since_last_order,up_order_rate_since_first_order
0,1,10,176.0,17.6,59,18,0.759259,196,10,1,10,1.400000,35791,0.582500,4.473875,0.776480,5.9,1.0,0,1.000000
1,1,10,176.0,17.6,59,18,0.759259,10258,9,2,10,3.333333,1946,0.552962,3.493716,0.713772,5.9,0.9,0,1.000000
2,1,10,176.0,17.6,59,18,0.759259,10326,1,5,5,5.000000,5526,0.521581,2.873635,0.652009,5.9,0.1,5,0.166667
3,1,10,176.0,17.6,59,18,0.759259,12427,10,1,10,3.300000,6476,0.529482,3.857058,0.740735,5.9,1.0,0,1.000000
4,1,10,176.0,17.6,59,18,0.759259,13032,3,2,10,6.333333,3751,0.479782,2.916796,0.657158,5.9,0.3,0,0.333333
5,1,10,176.0,17.6,59,18,0.759259,13176,2,2,5,6.000000,379450,0.711884,5.972111,0.832555,5.9,0.2,5,0.222222
6,1,10,176.0,17.6,59,18,0.759259,14084,1,1,1,2.000000,15935,0.629150,5.290505,0.810982,5.9,0.1,9,0.100000
7,1,10,176.0,17.6,59,18,0.759259,17122,1,5,5,6.000000,13880,0.527426,3.082390,0.675576,5.9,0.1,5,0.166667
8,1,10,176.0,17.6,59,18,0.759259,25133,8,3,10,4.000000,6196,0.580745,3.848447,0.740155,5.9,0.8,0,1.000000
9,1,10,176.0,17.6,59,18,0.759259,26088,2,1,2,4.500000,2523,0.404987,2.169390,0.539041,5.9,0.2,8,0.200000


## XG BOOST Data Prep

In [22]:
order_new_2 = order_new[['user_id','eval_set']]
order_new_2

,user_id,eval_set
0,1,prior
1,1,prior
2,1,prior
3,1,prior
4,1,prior
...,...,...
3421078,206209,prior
3421079,206209,prior
3421080,206209,prior
3421081,206209,prior


In [23]:
on2_train = order_new_2.loc[order_new_2['eval_set'] == 'train']
on2_test = order_new_2.loc[order_new_2['eval_set'] == 'Test']
order_new_3=pd.concat([on2_train, on2_test])
order_new_3

,user_id,eval_set
10,1,train
25,2,train
49,5,train
74,7,train
78,8,train
...,...,...
3420918,206202,Test
3420929,206204,Test
3421001,206206,Test
3421018,206207,Test


In [24]:
df_1 = pd.merge(data,order_new_3, on = 'user_id', how = 'left' )
df_1

,user_id,user_orders,sum_days_since_prior_order,avg_days_since_prior_order,total_products,distinct_products,reordered_ratio,product_id,up_orders,up_first_order,...,up_average_cart_position,prod_orders,prod_reorder_probability,prod_reorder_times,prod_reorder_ratio,user_average_basket,up_order_rate,up_orders_since_last_order,up_order_rate_since_first_order,eval_set
0,1,10,176.0,17.600000,59,18,0.759259,196,10,1,...,1.400000,35791,0.582500,4.473875,0.776480,5.900000,1.000000,0,1.000000,train
1,1,10,176.0,17.600000,59,18,0.759259,10258,9,2,...,3.333333,1946,0.552962,3.493716,0.713772,5.900000,0.900000,0,1.000000,train
2,1,10,176.0,17.600000,59,18,0.759259,10326,1,5,...,5.000000,5526,0.521581,2.873635,0.652009,5.900000,0.100000,5,0.166667,train
3,1,10,176.0,17.600000,59,18,0.759259,12427,10,1,...,3.300000,6476,0.529482,3.857058,0.740735,5.900000,1.000000,0,1.000000,train
4,1,10,176.0,17.600000,59,18,0.759259,13032,3,2,...,6.333333,3751,0.479782,2.916796,0.657158,5.900000,0.300000,0,0.333333,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13307948,206209,13,210.0,16.153846,129,68,0.525862,43961,3,4,...,8.000000,55371,0.514642,2.706967,0.630583,9.923077,0.230769,1,0.300000,train
13307949,206209,13,210.0,16.153846,129,68,0.525862,44325,1,7,...,8.000000,3485,0.289890,1.669861,0.401148,9.923077,0.076923,6,0.142857,train
13307950,206209,13,210.0,16.153846,129,68,0.525862,48370,1,11,...,8.000000,3934,0.592561,3.325444,0.699288,9.923077,0.076923,2,0.333333,train
13307951,206209,13,210.0,16.153846,129,68,0.525862,48697,1,7,...,6.000000,9783,0.260662,1.556811,0.357661,9.923077,0.076923,6,0.142857,train


In [25]:
Train = df_1[df_1.eval_set == 'train']
Test = df_1[df_1.eval_set == 'Test']

In [26]:
Train.head(10)

,user_id,user_orders,sum_days_since_prior_order,avg_days_since_prior_order,total_products,distinct_products,reordered_ratio,product_id,up_orders,up_first_order,...,up_average_cart_position,prod_orders,prod_reorder_probability,prod_reorder_times,prod_reorder_ratio,user_average_basket,up_order_rate,up_orders_since_last_order,up_order_rate_since_first_order,eval_set
0,1,10,176.0,17.6,59,18,0.759259,196,10,1,...,1.400000,35791,0.582500,4.473875,0.776480,5.9,1.0,0,1.000000,train
1,1,10,176.0,17.6,59,18,0.759259,10258,9,2,...,3.333333,1946,0.552962,3.493716,0.713772,5.9,0.9,0,1.000000,train
2,1,10,176.0,17.6,59,18,0.759259,10326,1,5,...,5.000000,5526,0.521581,2.873635,0.652009,5.9,0.1,5,0.166667,train
3,1,10,176.0,17.6,59,18,0.759259,12427,10,1,...,3.300000,6476,0.529482,3.857058,0.740735,5.9,1.0,0,1.000000,train
4,1,10,176.0,17.6,59,18,0.759259,13032,3,2,...,6.333333,3751,0.479782,2.916796,0.657158,5.9,0.3,0,0.333333,train
5,1,10,176.0,17.6,59,18,0.759259,13176,2,2,...,6.000000,379450,0.711884,5.972111,0.832555,5.9,0.2,5,0.222222,train
6,1,10,176.0,17.6,59,18,0.759259,14084,1,1,...,2.000000,15935,0.629150,5.290505,0.810982,5.9,0.1,9,0.100000,train
7,1,10,176.0,17.6,59,18,0.759259,17122,1,5,...,6.000000,13880,0.527426,3.082390,0.675576,5.9,0.1,5,0.166667,train
8,1,10,176.0,17.6,59,18,0.759259,25133,8,3,...,4.000000,6196,0.580745,3.848447,0.740155,5.9,0.8,0,1.000000,train
9,1,10,176.0,17.6,59,18,0.759259,26088,2,1,...,4.500000,2523,0.404987,2.169390,0.539041,5.9,0.2,8,0.200000,train


In [27]:
#Train_1 = pd.merge(Train,order_product_new, on = 'product_id', how = 'left')
#Train_1.head(10)

In [29]:
t=order_new[order_new['eval_set']=='train']
join=pd.merge(t,order_product_new,on='order_id',how='left')

In [30]:
order_new_t=pd.DataFrame({
    'user_id':join['user_id'],
    'product_id':join['product_id'],
    'reordered':join['reordered']
})

In [31]:
df_train=pd.merge(Train,order_new_t, on=['user_id','product_id'],how='left')

In [32]:
df_train.head(10)

,user_id,user_orders,sum_days_since_prior_order,avg_days_since_prior_order,total_products,distinct_products,reordered_ratio,product_id,up_orders,up_first_order,...,prod_orders,prod_reorder_probability,prod_reorder_times,prod_reorder_ratio,user_average_basket,up_order_rate,up_orders_since_last_order,up_order_rate_since_first_order,eval_set,reordered
0,1,10,176.0,17.6,59,18,0.759259,196,10,1,...,35791,0.582500,4.473875,0.776480,5.9,1.0,0,1.000000,train,1.0
1,1,10,176.0,17.6,59,18,0.759259,10258,9,2,...,1946,0.552962,3.493716,0.713772,5.9,0.9,0,1.000000,train,1.0
2,1,10,176.0,17.6,59,18,0.759259,10326,1,5,...,5526,0.521581,2.873635,0.652009,5.9,0.1,5,0.166667,train,NaN
3,1,10,176.0,17.6,59,18,0.759259,12427,10,1,...,6476,0.529482,3.857058,0.740735,5.9,1.0,0,1.000000,train,NaN
4,1,10,176.0,17.6,59,18,0.759259,13032,3,2,...,3751,0.479782,2.916796,0.657158,5.9,0.3,0,0.333333,train,1.0
5,1,10,176.0,17.6,59,18,0.759259,13176,2,2,...,379450,0.711884,5.972111,0.832555,5.9,0.2,5,0.222222,train,NaN
6,1,10,176.0,17.6,59,18,0.759259,14084,1,1,...,15935,0.629150,5.290505,0.810982,5.9,0.1,9,0.100000,train,NaN
7,1,10,176.0,17.6,59,18,0.759259,17122,1,5,...,13880,0.527426,3.082390,0.675576,5.9,0.1,5,0.166667,train,NaN
8,1,10,176.0,17.6,59,18,0.759259,25133,8,3,...,6196,0.580745,3.848447,0.740155,5.9,0.8,0,1.000000,train,1.0
9,1,10,176.0,17.6,59,18,0.759259,26088,2,1,...,2523,0.404987,2.169390,0.539041,5.9,0.2,8,0.200000,train,1.0


In [33]:
df_train.drop('eval_set',axis=1,inplace=True)
df_test=Test.drop('eval_set',axis=1)

In [34]:
df_train=df_train.fillna(0)
df_test=df_test.fillna(0)

In [35]:
train_sample = df_train.sample(frac = 0.01,replace = False,random_state=1)
train_sample.head(10)
#from sklearn.utils import shuffle
#import numpy as np
#train_sample1=np.random.shuffle(train_sample)

,user_id,user_orders,sum_days_since_prior_order,avg_days_since_prior_order,total_products,distinct_products,reordered_ratio,product_id,up_orders,up_first_order,...,up_average_cart_position,prod_orders,prod_reorder_probability,prod_reorder_times,prod_reorder_ratio,user_average_basket,up_order_rate,up_orders_since_last_order,up_order_rate_since_first_order,reordered
7839505,190548,20,257.0,12.850000,232,124,0.486486,31766,1,9,...,10.00,8483,0.500885,3.002832,0.666981,11.600000,0.050000,11,0.083333,0.0
211790,5349,5,49.0,9.800000,49,30,0.542857,31126,1,3,...,4.00,712,0.390071,2.524823,0.603933,9.800000,0.200000,2,0.333333,0.0
8269606,201184,10,138.0,13.800000,139,55,0.694215,7021,4,2,...,10.75,17629,0.334048,1.718394,0.418061,13.900000,0.400000,0,0.444444,0.0
5350149,129829,11,75.0,6.818182,267,148,0.455939,4347,1,8,...,12.00,3637,0.438053,2.298989,0.565026,24.272727,0.090909,3,0.250000,0.0
8392658,204183,40,304.0,7.600000,285,113,0.618705,19678,1,36,...,7.00,32534,0.471232,2.581859,0.612682,7.125000,0.025000,4,0.200000,0.0
759058,18485,12,243.0,20.250000,100,55,0.489130,38028,1,12,...,10.00,7004,0.404858,2.181252,0.541548,8.333333,0.083333,0,1.000000,1.0
7149883,173850,99,264.0,2.666667,841,248,0.721411,14252,1,97,...,10.00,2096,0.384876,2.365688,0.577290,8.494949,0.010101,2,0.333333,0.0
7379539,179499,34,210.0,6.176471,194,76,0.627660,26940,1,8,...,6.00,19286,0.430035,2.219333,0.549414,5.705882,0.029412,26,0.037037,0.0
2401935,58626,10,130.0,13.000000,263,142,0.512712,39618,8,1,...,10.00,529,0.404651,2.460465,0.593573,26.300000,0.800000,2,0.800000,1.0
8304867,202049,49,355.0,7.244898,610,237,0.620632,7478,1,36,...,3.00,533,0.121951,1.181818,0.153846,12.448980,0.020408,13,0.071429,0.0


In [36]:
test_sample = df_test.sample(frac = 0.01,replace = False,random_state=1)
test_sample

,user_id,user_orders,sum_days_since_prior_order,avg_days_since_prior_order,total_products,distinct_products,reordered_ratio,product_id,up_orders,up_first_order,up_last_order,up_average_cart_position,prod_orders,prod_reorder_probability,prod_reorder_times,prod_reorder_ratio,user_average_basket,up_order_rate,up_orders_since_last_order,up_order_rate_since_first_order
9799037,151684,16,224.0,14.000000,160,123,0.241830,2838,1,3,3,5.0,2019,0.285714,1.584772,0.368995,10.000000,0.062500,13,0.071429
2110863,32845,3,36.0,12.000000,38,30,0.285714,2596,1,1,1,2.0,2574,0.285071,1.656371,0.396270,12.666667,0.333333,2,0.333333
2038351,31734,9,55.0,6.111111,147,34,0.911290,12986,6,1,6,15.0,1388,0.390000,1.982857,0.495677,16.333333,0.666667,3,0.666667
3967730,61526,41,354.0,8.634146,893,202,0.797921,7500,1,28,28,20.0,9450,0.183932,1.315789,0.240000,21.780488,0.024390,13,0.071429
3797943,58921,5,91.0,18.200000,56,46,0.200000,43876,1,1,1,3.0,871,0.279297,1.701172,0.412170,11.200000,0.200000,4,0.200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11581615,179423,3,60.0,20.000000,64,46,0.545455,16198,1,1,1,31.0,1356,0.355755,2.026906,0.506637,21.333333,0.333333,2,0.333333
10902996,168857,9,129.0,14.333333,275,157,0.473896,1006,2,2,5,16.5,3767,0.375417,2.095106,0.522697,30.555556,0.222222,4,0.250000
1912581,29840,36,340.0,9.444444,368,201,0.469101,26209,5,5,26,8.4,140627,0.557770,3.134867,0.681007,10.222222,0.138889,10,0.156250
4351711,67426,16,79.0,4.937500,135,96,0.293233,46918,1,7,7,19.0,130,0.255556,1.444444,0.307692,8.437500,0.062500,9,0.100000


In [37]:
X_train = train_sample.drop('reordered', axis = 1)
Y_train = train_sample['reordered']

In [43]:
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
import sklearn.metrics as metrics
from sklearn.metrics import confusion_matrix, accuracy_score

In [44]:
data_dmatrix = xgb.DMatrix(data=X_train,label=Y_train)

In [51]:
param_grid = {'eta': [0.1,0.2, 0.3],
              'max_depth': [5, 6, 7],
              #'colsample_bytree': [0.5,0.7,1],
              'objective': "binary:logistic",
              'eval_metric': "auc"
             }

In [52]:
import itertools as it
parameters = it.product(*(param_grid[title] for title in list(param_grid)))
print(list(param_grid))

['eta', 'max_depth', 'objective', 'eval_metric']


In [53]:
def cross_validation_split(dataset, folds):
        dataset_split = []
        fold_size = int(X_train.shape[0] / folds) #number of rows in each fold
        ini_index=[0] #start from index 0
        remains= fold_size % folds 
        
        
        # for loop to save each fold
        for i in range(1,remains):
            ini_index.append(i*fold_size+1)

        for j in range (remains, folds+1):
            ini_index.append(j*fold_size+remains)

            # save the fold     
        for n in range(len(ini_index)-1):
            dataset_split.append([ini_index[n], ini_index[n+1]])
            
        return dataset_split

In [54]:
index=cross_validation_split(X_train,5)
index

[[0, 16950], [16950, 33899], [33899, 50848], [50848, 67800], [67800, 84749]]

In [55]:
def xg_boost(X_train, Y_train,param,n_round,es_round, k):
    index=cross_validation_split(X_train,k)
    
    br = []
    score=[]
    for i in range(k):
        
        cv_xtest = X_train.iloc[index[i][0]:index[i][1],:]
        cv_ytest = Y_train.iloc[index[i][0]:index[i][1],]
    
        cv_xtrain = X_train.drop(X_train.index[index[i][0]:index[i][1]])
        cv_ytrain = Y_train.drop(X_train.index[index[i][0]:index[i][1]])
        
        d_train = xgb.DMatrix(data = cv_xtrain, label = cv_ytrain)
        d_test = xgb.DMatrix(data = cv_xtest, label = cv_ytest)
        xgb_train = xgb.train(params = param,dtrain = d_train,num_boost_round = n_round,
                              early_stopping_rounds = es_round,
                              evals=[(d_train,'train'),(d_test, 'test')])
        
        #use auc to evaluate 
        #fpr, tpr, threshold = roc_curve(cv_ytest, xgb_train.predict(d_test))
        #auc_roc = metrics.auc(fpr, tpr)
        #auc.append(auc_roc)
        best_round = xgb_train.best_iteration
        value = xgb_train.best_score
        score.append(value)
        br.append(best_round)
    
    return np.mean(score),np.mean(br),best_round

In [ ]:
sc=[]
for i in list(parameters):
    param_opt = {'eta': i[0], 
                 'max_depth': i[1],
                 #'colsample_bytree': i[2],
                 'objective':'binary:logistic',
                 'eval_metric' : 'auc'
                }
    best_round = xg_boost(X_train, Y_train,param_opt, 2000, 50,5)
    sc.append(best_round)
    
print(best_round,i)

[0]	train-auc:0.81962	test-auc:0.81175
[1]	train-auc:0.82452	test-auc:0.81431
[2]	train-auc:0.82585	test-auc:0.81623
[3]	train-auc:0.82712	test-auc:0.81662
[4]	train-auc:0.82753	test-auc:0.81729
[5]	train-auc:0.82877	test-auc:0.81905
[6]	train-auc:0.82994	test-auc:0.81963
[7]	train-auc:0.83103	test-auc:0.82066
[8]	train-auc:0.83195	test-auc:0.82093
[9]	train-auc:0.83259	test-auc:0.82078
[10]	train-auc:0.83320	test-auc:0.82168
[11]	train-auc:0.83373	test-auc:0.82152
[12]	train-auc:0.83429	test-auc:0.82160
[13]	train-auc:0.83496	test-auc:0.82210
[14]	train-auc:0.83552	test-auc:0.82228
[15]	train-auc:0.83589	test-auc:0.82229
[16]	train-auc:0.83646	test-auc:0.82242
[17]	train-auc:0.83675	test-auc:0.82263
[18]	train-auc:0.83703	test-auc:0.82277
[19]	train-auc:0.83771	test-auc:0.82275
[20]	train-auc:0.83812	test-auc:0.82266
[21]	train-auc:0.83848	test-auc:0.82274
[22]	train-auc:0.83893	test-auc:0.82281
[23]	train-auc:0.83937	test-auc:0.82311
[24]	train-auc:0.84003	test-auc:0.82315
[25]	train

[104]	train-auc:0.86477	test-auc:0.83391
[105]	train-auc:0.86496	test-auc:0.83394
[106]	train-auc:0.86502	test-auc:0.83391
[107]	train-auc:0.86516	test-auc:0.83386
[108]	train-auc:0.86536	test-auc:0.83391
[109]	train-auc:0.86557	test-auc:0.83386
[110]	train-auc:0.86579	test-auc:0.83395
[111]	train-auc:0.86610	test-auc:0.83390
[0]	train-auc:0.81883	test-auc:0.80954
[1]	train-auc:0.82063	test-auc:0.81249
[2]	train-auc:0.82409	test-auc:0.81557
[3]	train-auc:0.82630	test-auc:0.81707
[4]	train-auc:0.82681	test-auc:0.81712
[5]	train-auc:0.82817	test-auc:0.81852
[6]	train-auc:0.82892	test-auc:0.81879
[7]	train-auc:0.83010	test-auc:0.81940
[8]	train-auc:0.83078	test-auc:0.81992
[9]	train-auc:0.83143	test-auc:0.82049
[10]	train-auc:0.83194	test-auc:0.82128
[11]	train-auc:0.83294	test-auc:0.82238
[12]	train-auc:0.83362	test-auc:0.82277
[13]	train-auc:0.83426	test-auc:0.82307
[14]	train-auc:0.83464	test-auc:0.82324
[15]	train-auc:0.83520	test-auc:0.82330
[16]	train-auc:0.83564	test-auc:0.82350
[1

[86]	train-auc:0.86070	test-auc:0.82839
[87]	train-auc:0.86098	test-auc:0.82844
[88]	train-auc:0.86112	test-auc:0.82849
[89]	train-auc:0.86137	test-auc:0.82854
[90]	train-auc:0.86157	test-auc:0.82856
[91]	train-auc:0.86171	test-auc:0.82855
[92]	train-auc:0.86179	test-auc:0.82848
[93]	train-auc:0.86210	test-auc:0.82856
[94]	train-auc:0.86220	test-auc:0.82860
[95]	train-auc:0.86234	test-auc:0.82863
[96]	train-auc:0.86253	test-auc:0.82865
[0]	train-auc:0.81861	test-auc:0.81191
[1]	train-auc:0.82173	test-auc:0.81371
[2]	train-auc:0.82416	test-auc:0.81593
[3]	train-auc:0.82529	test-auc:0.81821
[4]	train-auc:0.82600	test-auc:0.81915
[5]	train-auc:0.82784	test-auc:0.82116
[6]	train-auc:0.82799	test-auc:0.82185
[7]	train-auc:0.82918	test-auc:0.82306
[8]	train-auc:0.82977	test-auc:0.82390
[9]	train-auc:0.83039	test-auc:0.82383
[10]	train-auc:0.83064	test-auc:0.82388
[11]	train-auc:0.83225	test-auc:0.82481
[12]	train-auc:0.83277	test-auc:0.82497
[13]	train-auc:0.83346	test-auc:0.82509
[14]	train

[94]	train-auc:0.86417	test-auc:0.82493
[95]	train-auc:0.86458	test-auc:0.82483
[96]	train-auc:0.86467	test-auc:0.82482
[97]	train-auc:0.86477	test-auc:0.82477
[98]	train-auc:0.86499	test-auc:0.82478
[99]	train-auc:0.86541	test-auc:0.82473
[100]	train-auc:0.86556	test-auc:0.82464
[0]	train-auc:0.81700	test-auc:0.81931
[1]	train-auc:0.82325	test-auc:0.82381
[2]	train-auc:0.82302	test-auc:0.82438
[3]	train-auc:0.82374	test-auc:0.82486
[4]	train-auc:0.82535	test-auc:0.82592
[5]	train-auc:0.82604	test-auc:0.82664
[6]	train-auc:0.82671	test-auc:0.82683
[7]	train-auc:0.82781	test-auc:0.82739
[8]	train-auc:0.82891	test-auc:0.82799
[9]	train-auc:0.82997	test-auc:0.82842
[10]	train-auc:0.83076	test-auc:0.82888
[11]	train-auc:0.83184	test-auc:0.83025
[12]	train-auc:0.83247	test-auc:0.83046
[13]	train-auc:0.83332	test-auc:0.83121
[14]	train-auc:0.83381	test-auc:0.83122
[15]	train-auc:0.83417	test-auc:0.83120
[16]	train-auc:0.83446	test-auc:0.83119
[17]	train-auc:0.83497	test-auc:0.83117
[18]	trai

[86]	train-auc:0.86152	test-auc:0.82787
[87]	train-auc:0.86169	test-auc:0.82783
[88]	train-auc:0.86177	test-auc:0.82785
[89]	train-auc:0.86207	test-auc:0.82782
[90]	train-auc:0.86230	test-auc:0.82781
[91]	train-auc:0.86250	test-auc:0.82777
[92]	train-auc:0.86295	test-auc:0.82784
[93]	train-auc:0.86322	test-auc:0.82784
[94]	train-auc:0.86360	test-auc:0.82768
[95]	train-auc:0.86394	test-auc:0.82749
[96]	train-auc:0.86416	test-auc:0.82744
[97]	train-auc:0.86460	test-auc:0.82733
[98]	train-auc:0.86482	test-auc:0.82731
[99]	train-auc:0.86505	test-auc:0.82722
[100]	train-auc:0.86539	test-auc:0.82727
[101]	train-auc:0.86584	test-auc:0.82729
[102]	train-auc:0.86612	test-auc:0.82713
[103]	train-auc:0.86629	test-auc:0.82720
[104]	train-auc:0.86647	test-auc:0.82717
[105]	train-auc:0.86687	test-auc:0.82712
[106]	train-auc:0.86708	test-auc:0.82701
[107]	train-auc:0.86735	test-auc:0.82705
[108]	train-auc:0.86757	test-auc:0.82703
[109]	train-auc:0.86791	test-auc:0.82707
[110]	train-auc:0.86811	test-a

...

[32]	train-auc:0.87551	test-auc:0.82277
[33]	train-auc:0.87650	test-auc:0.82293
[34]	train-auc:0.87733	test-auc:0.82290
[35]	train-auc:0.87823	test-auc:0.82266
[36]	train-auc:0.87919	test-auc:0.82294
[37]	train-auc:0.87999	test-auc:0.82284
[38]	train-auc:0.88087	test-auc:0.82287
[39]	train-auc:0.88176	test-auc:0.82289
[40]	train-auc:0.88268	test-auc:0.82315
[41]	train-auc:0.88333	test-auc:0.82310
[42]	train-auc:0.88414	test-auc:0.82310
[43]	train-auc:0.88521	test-auc:0.82301
[44]	train-auc:0.88584	test-auc:0.82298
[45]	train-auc:0.88632	test-auc:0.82310
[46]	train-auc:0.88697	test-auc:0.82290
[47]	train-auc:0.88730	test-auc:0.82296
[48]	train-auc:0.88795	test-auc:0.82302
[49]	train-auc:0.88886	test-auc:0.82282
[50]	train-auc:0.88941	test-auc:0.82275
[51]	train-auc:0.88989	test-auc:0.82269
[52]	train-auc:0.89043	test-auc:0.82266
[53]	train-auc:0.89073	test-auc:0.82268
[54]	train-auc:0.89110	test-auc:0.82254
[55]	train-auc:0.89159	test-auc:0.82261
[56]	train-auc:0.89179	test-auc:0.82254


[48]	train-auc:0.88762	test-auc:0.82519
[49]	train-auc:0.88842	test-auc:0.82511
[50]	train-auc:0.88888	test-auc:0.82495
[51]	train-auc:0.88915	test-auc:0.82498
[52]	train-auc:0.88997	test-auc:0.82483
[53]	train-auc:0.89023	test-auc:0.82474
[54]	train-auc:0.89104	test-auc:0.82464
[55]	train-auc:0.89157	test-auc:0.82470
[56]	train-auc:0.89235	test-auc:0.82464
[57]	train-auc:0.89335	test-auc:0.82448
[58]	train-auc:0.89371	test-auc:0.82434
[59]	train-auc:0.89413	test-auc:0.82434
[60]	train-auc:0.89430	test-auc:0.82434
[61]	train-auc:0.89467	test-auc:0.82434
[62]	train-auc:0.89483	test-auc:0.82431
[63]	train-auc:0.89521	test-auc:0.82420
[64]	train-auc:0.89547	test-auc:0.82412
[65]	train-auc:0.89575	test-auc:0.82417
[66]	train-auc:0.89612	test-auc:0.82414
[67]	train-auc:0.89639	test-auc:0.82403
[68]	train-auc:0.89702	test-auc:0.82418
[69]	train-auc:0.89727	test-auc:0.82412
[70]	train-auc:0.89762	test-auc:0.82414
[0]	train-auc:0.82809	test-auc:0.81533
[1]	train-auc:0.83399	test-auc:0.82102
[2

[72]	train-auc:0.89418	test-auc:0.82572
[73]	train-auc:0.89438	test-auc:0.82576
[74]	train-auc:0.89497	test-auc:0.82574
[75]	train-auc:0.89545	test-auc:0.82565
[76]	train-auc:0.89574	test-auc:0.82553
[77]	train-auc:0.89624	test-auc:0.82557
[78]	train-auc:0.89663	test-auc:0.82555
[79]	train-auc:0.89679	test-auc:0.82547
[80]	train-auc:0.89692	test-auc:0.82543
[81]	train-auc:0.89703	test-auc:0.82538
[82]	train-auc:0.89713	test-auc:0.82540
[83]	train-auc:0.89770	test-auc:0.82542
[84]	train-auc:0.89809	test-auc:0.82549
[85]	train-auc:0.89837	test-auc:0.82544
[86]	train-auc:0.89869	test-auc:0.82538
[87]	train-auc:0.89952	test-auc:0.82523
[88]	train-auc:0.89977	test-auc:0.82516
[0]	train-auc:0.82993	test-auc:0.81391
[1]	train-auc:0.83471	test-auc:0.81720
[2]	train-auc:0.83726	test-auc:0.81823
[3]	train-auc:0.83868	test-auc:0.81767
[4]	train-auc:0.84058	test-auc:0.81922
[5]	train-auc:0.84109	test-auc:0.81947
[6]	train-auc:0.84293	test-auc:0.82078
[7]	train-auc:0.84448	test-auc:0.82107
[8]	trai

[98]	train-auc:0.90950	test-auc:0.82952
[0]	train-auc:0.83038	test-auc:0.81440
[1]	train-auc:0.83422	test-auc:0.81627
[2]	train-auc:0.83647	test-auc:0.81719
[3]	train-auc:0.83862	test-auc:0.81851
[4]	train-auc:0.84059	test-auc:0.81989
[5]	train-auc:0.84111	test-auc:0.81964
[6]	train-auc:0.84252	test-auc:0.82030
[7]	train-auc:0.84354	test-auc:0.82058
[8]	train-auc:0.84511	test-auc:0.82085
[9]	train-auc:0.84671	test-auc:0.82220
[10]	train-auc:0.84826	test-auc:0.82254
[11]	train-auc:0.84956	test-auc:0.82310
[12]	train-auc:0.85124	test-auc:0.82420
[13]	train-auc:0.85259	test-auc:0.82446
[14]	train-auc:0.85361	test-auc:0.82474
[15]	train-auc:0.85489	test-auc:0.82462
[16]	train-auc:0.85622	test-auc:0.82499
[17]	train-auc:0.85726	test-auc:0.82517
[18]	train-auc:0.85895	test-auc:0.82520
[19]	train-auc:0.86067	test-auc:0.82546
[20]	train-auc:0.86195	test-auc:0.82555
[21]	train-auc:0.86313	test-auc:0.82578
[22]	train-auc:0.86411	test-auc:0.82559
[23]	train-auc:0.86509	test-auc:0.82525
[24]	train

[23]	train-auc:0.86318	test-auc:0.82603
[24]	train-auc:0.86400	test-auc:0.82639
[25]	train-auc:0.86519	test-auc:0.82601
[26]	train-auc:0.86647	test-auc:0.82581
[27]	train-auc:0.86710	test-auc:0.82582
[28]	train-auc:0.86811	test-auc:0.82584
[29]	train-auc:0.86934	test-auc:0.82595
[30]	train-auc:0.87018	test-auc:0.82599
[31]	train-auc:0.87119	test-auc:0.82631
[32]	train-auc:0.87215	test-auc:0.82639
[33]	train-auc:0.87327	test-auc:0.82628
[34]	train-auc:0.87411	test-auc:0.82612
[35]	train-auc:0.87521	test-auc:0.82610
[36]	train-auc:0.87639	test-auc:0.82626
[37]	train-auc:0.87723	test-auc:0.82636
[38]	train-auc:0.87809	test-auc:0.82666
[39]	train-auc:0.87888	test-auc:0.82635
[40]	train-auc:0.87972	test-auc:0.82635
[41]	train-auc:0.88048	test-auc:0.82606
[42]	train-auc:0.88142	test-auc:0.82631
[43]	train-auc:0.88198	test-auc:0.82627
[44]	train-auc:0.88311	test-auc:0.82603
[45]	train-auc:0.88374	test-auc:0.82599
[46]	train-auc:0.88428	test-auc:0.82603
[47]	train-auc:0.88496	test-auc:0.82596


In [ ]:
best_param = {'objective':'binary:logistic','eta': 0.1,'max_depth': 5, 'colsample_bytree': 0.5,'eval_metric':'auc'}

In [ ]:
xg_reg = xgb.train(params=best_param, dtrain=data_dmatrix, num_boost_round=180)

In [ ]:
pred=xg_reg.predict(data_dmatrix)
test_matrix= xgb.DMatrix(test_sample)
ypreds_proba = xg_reg.predict(test_matrix)

In [ ]:
[fpr, tpr, thr]= roc_curve(Y_train,pred)
plt.figure()
plt.plot(fpr, tpr)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.show()

In [ ]:
print("AUC: %2.3f" % auc(fpr, tpr))

In [ ]:
xgb.plot_importance(xg_reg)
plt.rcParams['figure.figsize'] = [5, 5]
plt.show()

In [ ]:
train_proba = pd.DataFrame({'reorder_probability' : pred})
train_proba

In [ ]:
acc=[]
pre=[]
rec=[]
for i in np.arange(0, 1,0.1):
    def filter_prob(p):
        if p['reorder_probability'] >= i:
            return "1"
        else:
            return "0"
    train_proba['Survived_predict']= train_proba.apply(filter_prob,axis=1)
    #print(train_proba.Survived_predict)
    #print(Y_train)
    cm = confusion_matrix(Y_train,train_proba.Survived_predict.astype('int'))
    #Accuracy (all correct / all) = TP + TN /all
    accuracy = (cm[0][0]+cm[1][1])/891
    #print(accuracy)
    #precision =TP/(TP+FP)
    precision=cm[0][0]/(cm[0][0]+cm[0][1])
    #recall=TP/(TP+FN)
    recall=cm[0][0]/(cm[0][0]+cm[1][1])
    acc.append(accuracy)
    pre.append(pre)
    rec.append(recall)
    
print(i,accuracy.max())
print(i,precision.max())
print(i,recall.max())

In [ ]:
test_table = pd.DataFrame({'test_reorder_probability' : ypreds_proba})
test_table

In [ ]:
def filter_prob(p):
        if p['test_reorder_probability'] >= 0.9:
            return "1"
        else:
            return "0"
test_table['reorder_predict']= test_table.apply(filter_prob,axis=1)